In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/myProject/Horse or Human

/content/drive/MyDrive/myProject/Horse or Human


In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

In [ ]:
train_folder = './horse-or-human'

In [ ]:
!unzip -q horse-or-human.zip -d $train_folder

In [ ]:
valid_folder = './validation-horse-or-human'

In [ ]:
!unzip -q validation-horse-or-human.zip -d $valid_folder

In [ ]:
ls

horse-or-human/     Untitled0.ipynb             validation-horse-or-human.zip
horse-or-human.zip  validation-horse-or-human/


In [ ]:
train_horse_dir = './horse-or-human/horses'
train_human_dir = './horse-or-human/humans'
valid_horse_dir = './horse-or-human/horses'
valid_human_dir = './horse-or-human/humans'

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
model = Sequential()
model.add(Conv2D(16,(3,3), activation = 'relu', input_shape = (150, 150, 3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation= 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer= RMSprop(learning_rate= 0.001), metrics = ['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_generator = ImageDataGenerator(rescale = 1/255)
valid_generator = ImageDataGenerator(rescale = 1/255)

In [ ]:
generated_train_data = train_generator.flow_from_directory(
    train_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
generated_valid_data = valid_generator.flow_from_directory(
    valid_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


In [ ]:
history = model.fit(
    generated_train_data,
    steps_per_epoch = 8,
    epochs = 15,
    verbose = 1,
    validation_data = generated_valid_data,
    validation_steps = 8
)

Epoch 1/15
8/8 [==============================] - 391s 48s/step - loss: 3.1306 - acc: 0.5984 - val_loss: 0.5126 - val_acc: 0.7109
Epoch 2/15
8/8 [==============================] - 8s 1s/step - loss: 0.6615 - acc: 0.7334
Epoch 3/15
8/8 [==============================] - 7s 1s/step - loss: 0.3462 - acc: 0.8632
Epoch 4/15
8/8 [==============================] - 7s 889ms/step - loss: 0.1213 - acc: 0.9611
Epoch 5/15
8/8 [==============================] - 8s 953ms/step - loss: 0.2066 - acc: 0.9155
Epoch 6/15
8/8 [==============================] - 8s 916ms/step - loss: 0.1612 - acc: 0.9355
Epoch 7/15
8/8 [==============================] - 8s 946ms/step - loss: 0.0418 - acc: 0.9822
Epoch 8/15
8/8 [==============================] - 9s 1s/step - loss: 0.0219 - acc: 0.9922
Epoch 9/15
8/8 [==============================] - 8s 912ms/step - loss: 0.4937 - acc: 0.8810
Epoch 10/15
8/8 [==============================] - 8s 1s/step - loss: 0.1056 - acc: 0.9644
Epoch 11/15
8/8 [===========================

In [ ]:
import numpy as np 
from google.colab import files
from keras.preprocessing import image 

upload = files.upload()

for fn in upload.keys():

  path = './' + fn
  img = image.load_img(path, target_size= (150, 150))
  x = image.img_to_array(img)/255

  x = np.expand_dims(x, axis= 0)
  
  images = np.vstack([x])
  print(images.shape)

  classes = model.predict(images, batch_size= 10)

  if classes[0] > 0.5:
    print(fn + 'is a human')
  else:
    print(fn + 'is a horse')


Saving portrait-horse-260nw-1913554042.webp to portrait-horse-260nw-1913554042 (1).webp
(1, 150, 150, 3)
portrait-horse-260nw-1913554042.webpis a horse


In [ ]:
model.save('hoh.h5')